In [4]:
# pip install -q langchain langchain-community langchain_huggingface chromadb

In [5]:
# !pip install langchain langchain-community langchain_huggingface huggingface_hub chromadb --upgrade

# 1. Use langchain

In [6]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "Your-API-Key"

In [8]:
# os.environ.get("HUGGINGFACEHUB_API_TOKEN")

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub

from transformers import pipeline
from langchain.llms import HuggingFacePipeline

# 1. embeddings 모델 
# set Korean embedding and llm odel
hf_embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

# hf_llm = HuggingFaceHub(
#     repo_id="skt/kogpt2-base-v2",
#     model_kwargs={"task": "text-generation"} ## question-answering tasK X. text-generation
# )

# 2. LLM 모델 
# pipeline 생성 (gpt2 또는 한국어 모델로 교체 가능)
pipe = pipeline("text-generation", model="skt/kogpt2-base-v2", max_new_tokens=64) # 128

# LangChain에서 호환되는 HuggingFacePipeline로 감싸기
hf_llm = HuggingFacePipeline(pipeline=pipe)

C:\Users\human\AppData\Local\Temp\ipykernel_4456\3535075103.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
Device set to use cpu
C:\Users\human\AppData\Local\Temp\ipykernel_4456\3535075103.py:21: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.


## Crawl Wiki data

In [10]:
import requests
from langchain.schema import Document
from bs4 import BeautifulSoup

# for Wikipedia documents (EN, KO)

# from langchain_community.document_loaders import WikipediaLoader

# By default, English documents (https://en.wikipedia.org))
# def load_Wiki_docs(query):
#     loader = WikipediaLoader(query=query, load_max_docs=1) # need !pip install wikipedia
#     documents = loader.load()
    
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=1000,
#         chunk_overlap=200
#     )
#     splits = text_splitter.split_documents(documents)
    
#     return splits


# For Korean query, get results from Korean wikipedia website and crawl and parse results
def load_Korean_wiki_docs(흑백요리사):
    url = f"https://ko.wikipedia.org/wiki/{topic}"
    
    response = requests.get(url)
    response.raise_for_status()  # raise Exception when error occurs

    # HTML parsing and extract body contents
    soup = BeautifulSoup(response.text, 'html.parser')
    content = soup.find('div', {'class': 'mw-parser-output'})  # find div including body contents 
    
    # Extract contents
    paragraphs = content.find_all('p')
    text = "\n".join([p.get_text() for p in paragraphs])  # concat all context in <p> tags 
 
    # convert to Document object (required for LangChain)
    documents = [Document(page_content=text, metadata={"source": url})]
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=50, # 1000
        chunk_overlap=10 # 20
    )
    splits = text_splitter.split_documents(documents)
    
    return splits

## Create vector DB

In [11]:
def create_vectorstore(splits): 
    vectorstore = Chroma.from_documents(documents=splits, embedding=hf_embeddings)
    return vectorstore

In [12]:
topic = "흑백요리사"
# Load wikipedia documents for this topic
splits = load_Korean_wiki_docs(topic) 
# Create vectorstore with this fetched docs
vectorstore = create_vectorstore(splits)

In [13]:
splits

[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다.'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]')]

## Answer with RAG

In [14]:
def create_rag_chain(vectorstore):
    prompt_template = """문맥을 참고하여 질문에 정확하고 간결하게 답하십시오.
    문맥: {context}
    질문: {question}
    답변:"""
    
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    chain_type_kwargs = {"prompt": PROMPT}

    qa_chain = RetrievalQA.from_chain_type(
        llm=hf_llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs=chain_type_kwargs,
        return_source_documents=True
    )
    
    return qa_chain

In [15]:
# create langchang RAG chain

qa_chain = create_rag_chain(vectorstore)

In [16]:
question = "심사위원을 누가 맡았어?"

# result = qa_chain({"query": question})
result = qa_chain.invoke({"query": question})

print ("결과:")
print(result["result"])

print("===============================")
print("출처:")
for doc in result["source_documents"]:
    print(doc.page_content)
    print("---")

결과:
문맥을 참고하여 질문에 정확하고 간결하게 답하십시오.
    문맥: 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]

《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의

넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를

대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다.
    질문: 심사위원을 누가 맡았어?
    답변: 심사위원은 이다
(중략 ) 멘트: 심사의 난이도별로 선정하였다.
( 중략 )
심사 대상
심사위원별 명단이 정리 정리되어 있다.
이 내용은 아래와 같다.
심사위원 : 셰프가 가장 선호하는 레시피로 선정되었다.
이런 레시피를 소개할 때가 있는데, 조리 과정이 다소
출처:
출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]
---
《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의
---
넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를
---
대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다.
---


In [17]:
docs = vectorstore.as_retriever().get_relevant_documents(question)
docs

C:\Users\human\AppData\Local\Temp\ipykernel_4456\2742417880.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = vectorstore.as_retriever().get_relevant_documents(question)


[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다.')]

In [18]:
docs = vectorstore.similarity_search(question, k=4)
docs

[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다.')]

In [19]:
# It seems vectorDB loading from embedding model works fine, but seems llm model does not.
# Some Korean llm model seems to work fine in text-generation task, but for Question-Ansering task, we might need another approach.

# 2. Use QA pipeline

In [20]:
# import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# Load model and tokenizer
model_name = "yjgwak/klue-bert-base-finetuned-squard-kor-v1"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set Q_A pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

Device set to use cpu


In [21]:
# Example: define question and context 
question = "오늘 날씨 어때?"
context = "오늘의 날씨는 맑고 따뜻한 기온이 유지될 것으로 보입니다."

# model chain
result = qa_pipeline(question=question, context=context)

# Result
print("질문:", question)
print("답변:", result['answer'])

질문: 오늘 날씨 어때?
답변: 맑고 따뜻한 기온이


## Answer with RAG

In [22]:
# search context in VectorStore
def retrieve_context(question, vectorstore):
    docs = vectorstore.similarity_search(question, k=4)
    if docs:
        return " ".join([doc.page_content for doc in docs])
        # return docs[0].page_content  # return first relevant doc
    else:
        return None

# Generate answer based on query and searched context similar to RAG chain
def answer_question_with_context(question, vectorstore):
    context = retrieve_context(question, vectorstore)
    if context:
        result = qa_pipeline(question=question, context=context)
        return result['answer'], context  # return answer and used source doc
    else:
        return "관련 문맥을 찾지 못했습니다.", None

In [23]:
# Example
question = "심사위원을 누가 맡았어?"

answer, used_context = answer_question_with_context(question, vectorstore)

print("질문:", question)
print("답변:", answer)
print("사용된 문맥:", used_context)

질문: 심사위원을 누가 맡았어?
답변: 백종원과 안성재가
사용된 문맥: 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1] 《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다.


# 3. Use Gemini+RAG

In [24]:
# It seems the best and simple and cost-free option when OpenAI api cannot be used.

In [25]:
#!pip install -q langchain langchain-community langchain_huggingface faiss-cpu google-generativeai #chromadb

In [26]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import google.generativeai as genai
import os

## RAG 없는 LLM 사용

In [ ]:
# 1. Gemini model
genai_api_key = "YOUR-API-KEY"
genai.configure(api_key=genai_api_key)

gemini_model = genai.GenerativeModel('gemini-1.5-flash')

In [28]:
prompt = f"""문맥을 바탕으로 답변
\n문맥: '+' 연산의 경우 X+y = 10X+y 다 
\n질문: 1+1? 
\n대답:"""
response = gemini_model.generate_content(prompt)
answer = response.candidates[0].content.parts[0].text
print(answer)

1 + 1 = 11



## VectorDB 에 데이터 저장

In [ ]:
# set Korean embedding and llm model
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "Your-API-Key" 
# hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
hf_embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask") 

In [30]:
# from langchain.vectorstores import Chroma
# # sample docs
# docs = [
#     Document(page_content="한국어 챗봇은 자연어 처리 기술을 사용하여 사용자와 대화를 나눕니다.", metadata={"source": "doc1"}),
#     Document(page_content="인공지능을 활용한 챗봇은 여러 산업에서 사용되고 있습니다.", metadata={"source": "doc2"}),
#     Document(page_content="한국어와 영어를 동시에 지원하는 챗봇이 점점 늘어나고 있습니다.", metadata={"source": "doc3"}),
#     Document(page_content="챗봇은 고객 서비스를 개선하고 사용자 경험을 향상시키는 데 중요한 역할을 합니다.", metadata={"source": "doc4"})
# ]

# # to avoid collision with previous one
# persist_directory = "./new_chroma_db"
# vectorstore = Chroma.from_documents(docs, embedding=hf_embeddings, persist_directory="./chroma_db")

In [31]:
comments_list = [
    {
        "writer": "@박은정-t1t",
        "comments": "우아 영상의 퀄리티가 어마어마 하네요 이런건 좋아요 눌러야 함",
        "like": 184 
    },
    {
        "writer": "@GhosttownKOR",
        "comments": "1:17 밀면은 아니고 소면입니다ㅋㅋㅋ 다른 지역은 모르겠는데 부산은 돼지국밥 시키면 소면주는 집들이 있습니다! 밀가루로 만들었으니 밀면이라고 하면 할 말 없지만.. 보통 부산에서 밀면은 냉면처럼 시원한 그 밀면에만 사용합니다",
        "like": 89        
    },
    {
        "writer": "@후후-k2q",
        "comments": "진짜 야무지게 드시고 가셨네... 퀸님 맛집 추천은 진짜 믿고봐요ㅋㅋㅋㅋ",
        "like": 70        
    },    
    {
        "writer": "@zheen4385",
        "comments": "송정3대국밥은 저도 가서 실망했어요… ㅋㅋㅋㅋ 저야 부산사니까 저기 한 번 간다고 손해는 없지만 끼니 횟수가 정해져 있는 관광객 입장에선 매우 실망했을 수도 있다 생각해요",
        "like": 504        
    },    
    {
        "writer": "@고효성-q8g",
        "comments": "부산인인데 부산사람은 각자 마음속에 자신이 제일 좋아하는 국밥집이 하나씩 있다고 합니다 ㅋㅋㅋ 간짜장 좋아하는데 유원은 꼭 가보고싶네요 좋은 정보알아갑니다 감사합니다.",
        "like": 161        
    },    
    {
        "writer": "@jeonpodong-jumin",
        "comments": "드디어 백설대학을 가셨네요! 매장이전하고 많이 깔끔해졌네요 ㅎㅎ 참치김밥을 드셨는데 여기 기본김밥이 미친맛이예요!! 유부와 우엉조림이 찌이이이인한 고소한 맛이랍니다 그리고 사장님 처음에 보면 투빅하고 불친절하다고 느낄수 있는데 부산아재의 유쾌솔직한 모습을 보셨네요 부산에 맛집들 많으니까 더 오셔서 영상올려주세요~",
        "like": 9        
    },  
    {
        "writer": "@soothingcream",
        "comments": "부산사람들한테 돼지국밥은 집앞이나 동네에 돼지국밥이 최고인 경우가 많고 워낙 돼지국밥 종류가 다양해서 (농도, 부위 등등) 개취가 심한 음식중 하나라 맛없다고 단정지어 말하기엔 그렇지만 국밥집은 관광오는분들에게 알려진 곳은 거르는게 맞는것 같아요 고관은 그냥 동네 가성비 맛집이었는데 sns이런데 알려지면서 더 그냥저냥 되어버렸고 가끔 포장용 함박만 사서 집에서 해먹는게 맛있더라고요 무튼 sns 맛집은 거의 맛이 변했거나 맛없는 경우가 대부분입니다..",
        "like": 30        
    },  
    {
        "writer": "@겉돌이-s5w",
        "comments": "08:57  와 김밥 써는거 개신기하네 ㅋㅋㅋㅋ ㄹㅇ 첨보는데 저런 광경은 ㅋㅋㅋ",
        "like": 128        
    },  
    {
        "writer": "@BrandoDio-u1e",
        "comments": "1:57 바오하우스 13팀 대기면 좋게 가셨네요ㅠㅠ. 부산 토백인데도 갈 때 마다 웨이팅이 30팀 넘게 있어서 아직도 못먹은...",
        "like": 35        
    },  
    {
        "writer": "@kjpark9951",
        "comments": "백설대학...  거의 10년전부터 가끔 가던 곳인데... 장사가 잘되어서 그런지 가게를 옮겨서 그런지... 사장님 쫌...  카드되냐고 물으니 계좌이체된다고 답하시더군요. 마법의 문장입니다.",
        "like": 27        
    },  
    {
        "writer": "@콩-k3z",
        "comments": "부산 현지인인데 동영상 다 보기도 전에 송정 국밥집 보고 이마 짚었습니다 ..😵‍💫 다음에 부산 오시면 송정국밥집 옆에 수영 본가국밥 가셔서 오겹국밥 셀프 코너에서 청양고추 몇개 가져와서 잘게 다진 후 국밥에 섞어서 꼭 드셔보세요! 서면시장 국밥 라인에선 수영이 젤 맛있고 양 많아용",
        "like": 7        
    },  
    {
        "writer": "@minsim8275",
        "comments": "부산국밥은 진짜 각양각색입니다 조금씩 다 다달라요 각자의 최애가 존재합니다 ㅎㅎ 저의 최애는 지금은 폐업한 용호동 국밥집,,과,,, 경성대 형제 돼지국밥, 영진돼지국밥입니다(본점)",
        "like": 43       
    }
]

print(len(comments_list))

12


In [32]:
for doc in comments_list:
    print(doc)

{'writer': '@박은정-t1t', 'comments': '우아 영상의 퀄리티가 어마어마 하네요 이런건 좋아요 눌러야 함', 'like': 184}
{'writer': '@GhosttownKOR', 'comments': '1:17 밀면은 아니고 소면입니다ㅋㅋㅋ 다른 지역은 모르겠는데 부산은 돼지국밥 시키면 소면주는 집들이 있습니다! 밀가루로 만들었으니 밀면이라고 하면 할 말 없지만.. 보통 부산에서 밀면은 냉면처럼 시원한 그 밀면에만 사용합니다', 'like': 89}
{'writer': '@후후-k2q', 'comments': '진짜 야무지게 드시고 가셨네... 퀸님 맛집 추천은 진짜 믿고봐요ㅋㅋㅋㅋ', 'like': 70}
{'writer': '@zheen4385', 'comments': '송정3대국밥은 저도 가서 실망했어요… ㅋㅋㅋㅋ 저야 부산사니까 저기 한 번 간다고 손해는 없지만 끼니 횟수가 정해져 있는 관광객 입장에선 매우 실망했을 수도 있다 생각해요', 'like': 504}
{'writer': '@고효성-q8g', 'comments': '부산인인데 부산사람은 각자 마음속에 자신이 제일 좋아하는 국밥집이 하나씩 있다고 합니다 ㅋㅋㅋ 간짜장 좋아하는데 유원은 꼭 가보고싶네요 좋은 정보알아갑니다 감사합니다.', 'like': 161}
{'writer': '@jeonpodong-jumin', 'comments': '드디어 백설대학을 가셨네요! 매장이전하고 많이 깔끔해졌네요 ㅎㅎ 참치김밥을 드셨는데 여기 기본김밥이 미친맛이예요!! 유부와 우엉조림이 찌이이이인한 고소한 맛이랍니다 그리고 사장님 처음에 보면 투빅하고 불친절하다고 느낄수 있는데 부산아재의 유쾌솔직한 모습을 보셨네요 부산에 맛집들 많으니까 더 오셔서 영상올려주세요~', 'like': 9}
{'writer': '@soothingcream', 'comments': '부산사람들한테 돼지국밥은 집앞이나 동네에 돼지국밥이 최고인 경우가 많고 워낙 돼지국밥 종류가 다양해서 (농도, 부위 등등) 개취가

In [33]:
documents = []
for doc in comments_list:
    each = Document(page_content=doc["comments"], 
                    metadata={"writer": doc["writer"], "like": doc["like"]})
    documents.append(each)
    

In [34]:
# convert to Document object (required for LangChain)
documents = [Document(page_content=doc["comments"], 
                    metadata={"writer": doc["writer"], "like": doc["like"]}) 
             for doc in comments_list]

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=200
# )
# splits = text_splitter.split_documents(documents)    

vectorstore = FAISS.from_documents(documents=documents, embedding=hf_embeddings)

## VectorDB로 쿼리와 유사한 문서 검색

In [35]:
query = "계좌이체"

results = vectorstore.similarity_search_with_score(query, k=4) # similarity_search_with_score(query=query, k=4)

for doc, score in results:
    print(f"* 유사도: [SIM={score:3f}]\n댓글: {doc.page_content}\n메타데이터: {doc.metadata}]")
    print()

* 유사도: [SIM=136.265320]
댓글: 백설대학...  거의 10년전부터 가끔 가던 곳인데... 장사가 잘되어서 그런지 가게를 옮겨서 그런지... 사장님 쫌...  카드되냐고 물으니 계좌이체된다고 답하시더군요. 마법의 문장입니다.
메타데이터: {'writer': '@kjpark9951', 'like': 27}]

* 유사도: [SIM=196.744522]
댓글: 드디어 백설대학을 가셨네요! 매장이전하고 많이 깔끔해졌네요 ㅎㅎ 참치김밥을 드셨는데 여기 기본김밥이 미친맛이예요!! 유부와 우엉조림이 찌이이이인한 고소한 맛이랍니다 그리고 사장님 처음에 보면 투빅하고 불친절하다고 느낄수 있는데 부산아재의 유쾌솔직한 모습을 보셨네요 부산에 맛집들 많으니까 더 오셔서 영상올려주세요~
메타데이터: {'writer': '@jeonpodong-jumin', 'like': 9}]

* 유사도: [SIM=211.218658]
댓글: 부산 현지인인데 동영상 다 보기도 전에 송정 국밥집 보고 이마 짚었습니다 ..😵‍💫 다음에 부산 오시면 송정국밥집 옆에 수영 본가국밥 가셔서 오겹국밥 셀프 코너에서 청양고추 몇개 가져와서 잘게 다진 후 국밥에 섞어서 꼭 드셔보세요! 서면시장 국밥 라인에선 수영이 젤 맛있고 양 많아용
메타데이터: {'writer': '@콩-k3z', 'like': 7}]

* 유사도: [SIM=213.062714]
댓글: 1:57 바오하우스 13팀 대기면 좋게 가셨네요ㅠㅠ. 부산 토백인데도 갈 때 마다 웨이팅이 30팀 넘게 있어서 아직도 못먹은...
메타데이터: {'writer': '@BrandoDio-u1e', 'like': 35}]



## RAG + LLM

In [ ]:
# RAG using prompt
def rag_chatbot(question):
    context_doc = vectorstore.similarity_search(question, k=3 ) # filter={"like": {"$gte": 10}}
    
    # context = " ".join([doc.page_content for doc in context_doc])
    context = "\n".join([f"문서{i+1}: " + doc.page_content for i, doc in enumerate(context_doc)])

    prompt = f"""Context에 기반해서 짧게 대답해줘
    Context: {context}\
    질문: {question}
    답변: """
    
    # response = gemini_model(prompt)    
    response = gemini_model.generate_content(prompt)
    answer = response.candidates[0].content.parts[0].text

    return answer, context

In [41]:
# sample question
question = "백설대학 계좌이체 가능해?"
response, context = rag_chatbot(question)

print("질문:", question)
print("답변:", response)
print('='*100)
print("출처 문서:", context)

질문: 백설대학 계좌이체 가능해?
답변: 문서1에 따르면 백설대학은 계좌이체 가능하다고 합니다.

출처 문서: 문서1: 백설대학...  거의 10년전부터 가끔 가던 곳인데... 장사가 잘되어서 그런지 가게를 옮겨서 그런지... 사장님 쫌...  카드되냐고 물으니 계좌이체된다고 답하시더군요. 마법의 문장입니다.
문서2: 드디어 백설대학을 가셨네요! 매장이전하고 많이 깔끔해졌네요 ㅎㅎ 참치김밥을 드셨는데 여기 기본김밥이 미친맛이예요!! 유부와 우엉조림이 찌이이이인한 고소한 맛이랍니다 그리고 사장님 처음에 보면 투빅하고 불친절하다고 느낄수 있는데 부산아재의 유쾌솔직한 모습을 보셨네요 부산에 맛집들 많으니까 더 오셔서 영상올려주세요~
문서3: 1:57 바오하우스 13팀 대기면 좋게 가셨네요ㅠㅠ. 부산 토백인데도 갈 때 마다 웨이팅이 30팀 넘게 있어서 아직도 못먹은...
문서4: 부산사람들한테 돼지국밥은 집앞이나 동네에 돼지국밥이 최고인 경우가 많고 워낙 돼지국밥 종류가 다양해서 (농도, 부위 등등) 개취가 심한 음식중 하나라 맛없다고 단정지어 말하기엔 그렇지만 국밥집은 관광오는분들에게 알려진 곳은 거르는게 맞는것 같아요 고관은 그냥 동네 가성비 맛집이었는데 sns이런데 알려지면서 더 그냥저냥 되어버렸고 가끔 포장용 함박만 사서 집에서 해먹는게 맛있더라고요 무튼 sns 맛집은 거의 맛이 변했거나 맛없는 경우가 대부분입니다..


## 목적에 맞게 Prompt engineering: 요약하기, 해시태그 생성 등

In [38]:
query = "송정3대국밥"
context_doc = vectorstore.similarity_search(query, k=3)    

context = "\n".join([f"문서{i+1}: " + doc.page_content for i, doc in enumerate(context_doc)])
prompt = f"Context:\n{context}\n\n주어진 정보를 기반으로 중요한 정보만을 추출해서 해시태그 형태로 list에 담아 한 줄로 리턴해줘. hashtags="
# prompt = f"Context:\n{context}\n\n주어진 정보를 기반으로 간단하게 요약해줘."

response = gemini_model.generate_content(prompt)
answer = response.candidates[0].content.parts[0].text

print("답변\n:", answer)
print("출처 문서:\n", context)

답변
: hashtags=["#부산맛집", "#돼지국밥", "#수영본가국밥", "#오겹국밥", "#개인취향존중", "#SNS맛집거르기", "#현지인추천", "#가성비맛집"]

출처 문서:
 문서1: 부산 현지인인데 동영상 다 보기도 전에 송정 국밥집 보고 이마 짚었습니다 ..😵‍💫 다음에 부산 오시면 송정국밥집 옆에 수영 본가국밥 가셔서 오겹국밥 셀프 코너에서 청양고추 몇개 가져와서 잘게 다진 후 국밥에 섞어서 꼭 드셔보세요! 서면시장 국밥 라인에선 수영이 젤 맛있고 양 많아용
문서2: 부산사람들한테 돼지국밥은 집앞이나 동네에 돼지국밥이 최고인 경우가 많고 워낙 돼지국밥 종류가 다양해서 (농도, 부위 등등) 개취가 심한 음식중 하나라 맛없다고 단정지어 말하기엔 그렇지만 국밥집은 관광오는분들에게 알려진 곳은 거르는게 맞는것 같아요 고관은 그냥 동네 가성비 맛집이었는데 sns이런데 알려지면서 더 그냥저냥 되어버렸고 가끔 포장용 함박만 사서 집에서 해먹는게 맛있더라고요 무튼 sns 맛집은 거의 맛이 변했거나 맛없는 경우가 대부분입니다..
문서3: 부산인인데 부산사람은 각자 마음속에 자신이 제일 좋아하는 국밥집이 하나씩 있다고 합니다 ㅋㅋㅋ 간짜장 좋아하는데 유원은 꼭 가보고싶네요 좋은 정보알아갑니다 감사합니다.


In [39]:
# # Note:
# 1. vectorstore에 들어가는 임베딩 모델 (예: jhgan/ko-sroberta-multitask)의 성능 체크 및 한국어데이터에 최적화 되어 있을 것
# 2. LLM에 들어가는 prompt 엔지니어링

# 참고:
# https://python.langchain.com/docs/integrations/vectorstores/faiss/